In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

data = np.loadtxt('SCPUnion2.1_mu_vs_z.txt',dtype={'names': ('name','redshift','dmod','dmoderr'),'formats': ('S12', 'f4','f4','f4','f4')})
data.sort(order='redshift')
plt.errorbar(data['redshift'],data['dmod'],yerr=data['dmoderr'],fmt='none')
plt.xlabel('redshift')
plt.ylabel('supernova distance modulus')
plt.show()

In [ ]:
# Next, load the Planck cosmology, and plot it with the supernovae.
from astropy.cosmology import Planck15
from astropy.cosmology import WMAP5

dmod_planck = Planck15.distmod(data['redshift'])
dmod_wmap = WMAP5.distmod(data['redshift'])

plt.errorbar(data['redshift'],data['dmod'],yerr=data['dmoderr'],fmt=None)
plt.xlabel('redshift')
plt.ylabel('supernova distance modulus')
plt.plot(data['redshift'],dmod_planck,label='Planck15')
plt.plot(data['redshift'],dmod_wmap,label='WMAP5')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Write your likelihood function here.
# Call it 'logLike'

In [ ]:


# First up: Try an optimizer; just find the ML values of omega_matter and H0.
import scipy.optimize as op
nll = lambda *args: -logLike(*args)
bounds = [(10.,100.),(0.,1.0)]
result = op.minimize(nll, [75.0, 0.25],args=( data['redshift'], data['dmod'], data['dmoderr']),bounds=bounds)
print result

# Try plotting this result on top of the supernova data.


In [ ]:
# Define a sensible prior.

def logPrior( parameters, z, dm, err):
    # Add a prior here.
    return 0.
    
def logProb( parameters, z, dm, err):
    lp = logPrior(parameters, z, dm, err)
    if not np.isfinite(lp):
        return -np.inf
    return lp + logLike(parameters, z, dm, err)

# Now try sampling!
ndim, nwalkers = 2, 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, logProb, args=(data['redshift'], data['dmod'], data['dmoderr']))
sampler.run_mcmc(pos, 500)
samples = sampler.chain[:, 50:, :].reshape((-1, ndim))




In [ ]:
# Plot the results:
import corner
H0_true, Om_true = (67.74 , 0.2589)
f_true = logProb( (H0_true, Om_true), data['redshift'], data['dmod'], data['dmoderr'])
fig = corner.corner(samples, labels=["$H_0$", "$\Omega_m$"],
                      truths=[H0_true, Om_true])
fig.savefig("triangle.png")


In [ ]:
# Now, re-make the first plot, but with the new parameter values:
# Next, load the Planck cosmology.
from astropy.cosmology import Planck15
from astropy.cosmology import WMAP5

fit_cosmo = cosm.FlatwCDM(H0=np.mean(samples[:,0]), Om0 = np.mean(samples[:,1]))

dmod_planck = Planck15.distmod(data['redshift'])
dmod_fit = fit_cosmo.distmod(data['redshift'])

plt.errorbar(data['redshift'],data['dmod'],yerr=data['dmoderr'],fmt=None)
plt.xlabel('redshift')
plt.ylabel('supernova distance modulus')
plt.plot(data['redshift'],dmod_planck,label='Planck15')
plt.plot(data['redshift'],dmod_fit,label='fit')                          
plt.legend(loc='lower right')
plt.show()

In [ ]:
# That's really hard to see, so let's make the residual plot:
plt.errorbar(data['redshift'],data['dmod'] - dmod_planck/u.mag,yerr=data['dmoderr'],fmt='none')
plt.xlabel('redshift')
plt.ylabel('$\Delta$ supernova distance modulus')
plt.plot(data['redshift'],dmod_fit-dmod_planck,label='fit')                          
plt.plot(data['redshift'],np.zeros_like(data['redshift']),label='planck')
plt.legend(loc='lower right')
plt.ylim(-1.5,1.5)
plt.show()